# 5-bus Market simulation with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSimulations.jl supports simulations that consist of sequential optimization problems
where results from previous problems inform subsequent problems in a variety of ways. This
example demonstrates some of these capabilities to represent electricity market clearing.

## Dependencies and Data
First, let's create `System`s to represent the Day-Ahead and Real-Time market clearing
process with hourly, and 5-minute time series data, respectively.

### Modeling Packages

In [1]:
using PowerSystems
using PowerSimulations
using PowerSystemCaseBuilder

### Data management packages

In [2]:
using Dates
using DataFrames

### Optimization packages

In [3]:
using Cbc # mip solver
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)
using Ipopt # solver that supports duals
ipopt_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)

[ Info: Precompiling Ipopt [b6b21f68-93f8-5de0-b562-5493be1d77c9]


MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawParameter("print_level") => 0])

### 5-bus Data
The five bus system data here includes hourly day-ahead data, 5-minute real-time market
data, and 6-second actual data. We'll only use the hourly and 5-minute data for the
example simulations below, but the 6-second data is included for future development.

In [4]:
sys_DA = build_system(SIIPExampleSystems, "5_bus_matpower_DA")
sys_RT = build_system(SIIPExampleSystems, "5_bus_matpower_RT")

┌ Info: Building new system 5_bus_matpower_DA from raw data
└   sys_descriptor.raw_data = "/Users/cbarrows/.julia/packages/PowerSystemCaseBuilder/1xViZ/data/matpower/case5_re_uc.m"
[ Info: extending matpower format with data: areas 1x3
[ Info: extending matpower format with data: gen_name 7x4
[ Info: extending matpower format by appending matrix "gen_name" in to "gen"
[ Info: reversing the orientation of branch 6 (4, 3) to be consistent with other parallel branches
[ Info: the voltage setpoint on generator 4 does not match the value at bus 4
[ Info: the voltage setpoint on generator 1 does not match the value at bus 1
[ Info: the voltage setpoint on generator 5 does not match the value at bus 10
[ Info: the voltage setpoint on generator 2 does not match the value at bus 1
[ Info: the voltage setpoint on generator 3 does not match the value at bus 3
[ Info: removing 1 cost terms from generator 4: [4000.0, 0.0]
[ Info: removing 1 cost terms from generator 1: [1400.0, 0.0]
[ Info: removin

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Arc,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,6
2,Area,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
3,Bus,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
4,Line,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
5,LoadZone,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
6,PhaseShiftingTransformer,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
7,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3
8,RenewableDispatch,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
9,ThermalStandard,ThermalGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5


## `OperationsProblemTemplate`s

In [5]:
template_uc = template_unit_commitment()
template_ed = template_economic_dispatch()


Operations Problem Specification
Transmission: CopperPlatePowerModel
Devices Models: 

	Type: ThermalStandard
 	Formulation: ThermalDispatch

	Type: HydroDispatch
 	Formulation: HydroDispatchRunOfRiver

	Type: PowerLoad
 	Formulation: StaticPowerLoad

	Type: RenewableFix
 	Formulation: FixedOutput

	Type: RenewableDispatch
 	Formulation: RenewableFullDispatch

	Type: HydroEnergyReservoir
 	Formulation: HydroDispatchRunOfRiver

	Type: InterruptibleLoad
 	Formulation: InterruptiblePowerLoad

Branches Models: 

	Type: Line
 	Formulation: StaticBranch

	Type: TapTransformer
 	Formulation: StaticBranch

	Type: Transformer2W
 	Formulation: StaticBranch

	Type: HVDCLine
 	Formulation: HVDCDispatch

Services Models:

	Type: VariableReserve{ReserveDown}
 	Formulation: RangeReserve

	Type: VariableReserve{ReserveUp}
 	Formulation: RangeReserve



### Define the Simulation Sequence

In [6]:
problems = SimulationProblems(
    UC = OperationsProblem(
        template_uc,
        sys_DA,
        optimizer = solver,
        balance_slack_variables = true,
    ),
    ED = OperationsProblem(
        template_ed,
        sys_RT,
        optimizer = ipopt_solver,
        constraint_duals = [:CopperPlateBalance],
    ),
)

feedforward_chronologies = Dict(("UC" => "ED") => Synchronize(periods = 24))

feedforward = Dict(
    ("ED", :devices, :ThermalStandard) => SemiContinuousFF(
        binary_source_problem = ON,
        affected_variables = [ACTIVE_POWER],
    ),
)

#cache = Dict("UC" => [TimeStatusChange(ThermalStandard, PSI.ON)])
intervals = Dict("UC" => (Hour(24), Consecutive()), "ED" => (Hour(1), Consecutive()))

DA_RT_sequence = SimulationSequence(
    problems = problems,
    intervals = intervals,
    ini_cond_chronology = InterProblemChronology(),
    feedforward_chronologies = feedforward_chronologies,
    feedforward = feedforward,
)

Feed Forward Chronology
-----------------------

ED: SemiContinuousFF -> ThermalStandard

                     UC--┐ from : On
                         |
┌----┬----┬----┬----┬----┼----┬----┬----┬----┬----┬----┐
|    |    |    |    |    |    |    |    |    |    |    |
|    |    |    |    |    |    |    |    |    |    |    |
└─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED ... (x24) to : ["P"]

Initial Condition Chronology
----------------------------

1
|
|
2 --> 2 ... (x24)   


## `Simulation`

In [7]:
file_path = mkpath(joinpath(".", "5-bus-simulation"))
sim = Simulation(
    name = "5bus-test",
    steps = 1,
    problems = problems,
    sequence = DA_RT_sequence,
    simulation_folder = file_path,
)

Simulation()


### Build simulation

In [8]:
build!(sim)

┌ Info: 
│  ──────────────────────────────────────────────────────────────────────────────
│                                        Time                   Allocations      
│                                ──────────────────────   ───────────────────────
│        Tot / % measured:            2.44s / 100%             331MiB / 100%     
│ 
│  Section               ncalls     time   %tot     avg     alloc   %tot      avg
│  ──────────────────────────────────────────────────────────────────────────────
│  Build Simulation           1    2.44s   100%   2.44s    331MiB  100%    331MiB
│    Build Problems           1    2.43s   100%   2.43s    331MiB  100%    331MiB
│      Problem UC             1    1.31s  53.9%   1.31s    165MiB  49.9%   165MiB
│        ThermalStandard      1    654ms  26.9%   654ms   54.6MiB  16.5%  54.6MiB
│        Services             1    103ms  4.22%   103ms   11.3MiB  3.42%  11.3MiB
│        Build pre-step       1   54.8ms  2.25%  54.8ms   3.75MiB  1.13%  3.75MiB
│   

BuildStatus.BUILT = 0

### Execute simulation
```julia

In [9]:
execute!(sim, enable_progress_bar = false)

Presolve 703 (-2597) rows, 1591 (-5969) columns and 2926 (-11102) elements
Perturbing problem by 0.001% of 10 - largest nonzero change 0.00010950765 ( 54.949531%) - largest zero change 0.00010938892
0  Obj 0.0084891848 Primal inf 370.27927 (427)
89  Obj 12594.074 Primal inf 189.71042 (302)
178  Obj 25071.046 Primal inf 100.17492 (233)
267  Obj 30948.692 Primal inf 57.768287 (187)
356  Obj 31143.2 Primal inf 21.395868 (119)
445  Obj 31624.341 Primal inf 5.8748382 (51)
495  Obj 31907.354
Optimal - objective value 31907.325
After Postsolve, objective 31907.325, infeasibilities - dual 42318.909 (240), primal 0 (0)
Presolved model was optimal, full model needs cleaning up
0  Obj 31907.325
Optimal - objective value 31907.325
Optimal objective 31907.32482 - 495 iterations time 0.012, Presolve 0.01

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code 

RunStatus.SUCCESSFUL = 0

```

In [10]:
# Results

First we can load the result metadata
```julia
results = SimulationResults(sim);
uc_results = get_problem_results(results, "UC")
ed_results = get_problem_results(results, "ED");
```

Then we can read and examine the results of interest
```julia
prices = read_dual(ed_results, :CopperPlateBalance)
```

or if we want to look at the realized values
```julia
read_realized_duals(ed_results)[:CopperPlateBalance]
```

*note that in this simulation the prices are all equal to the balance slack
penalty value of $100000/MWh because there is unserved energy in the result*

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*